In [1]:
import optuna
from functools import partial

In [ ]:
def objective(trial, X, y):
    param = {
        'n_estimators': 2000,
        'max_depth': trial.suggest_int('max_depth', 3, 11),
        'learning_rate': trial.suggetst_uniform('learning_rate', 0.005, 0.01),
        'subsample': trial.suggest_categorical('subsample', list(range(0.4, 1,1, 0.1))),
        'colsample_bylevel': trial.suggest_categorical('colsample_bylevel', list(range(0.4, 1,1, 0.1))),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 100),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 100),
        'n_job': -1
    }
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    model = XGBRegressor(**param)
    train_scores, valid_scores = [], []
    for train_idx, valid_idx in kf.split(X_train):
        X_tmp_train, X_tmp_valid = X_train.iloc[train_idx, :], X_train.iloc[valid_idx, :]
        y_tmp_train, y_tmp_valid = y_train[train_idx], y_train[valid_idx]
        model = model.fit(X_tmp_train, y_tmp_train,
                         eval_set=[(X_tmp_valid, y_tmp_valid)], eval_metric=['rmse'],
                         early_stopping_rounds=30,
                         Callbacks=optuna.integration.XGBoostPruningCallback(trial, observation_key='validation_0-rmse'))
        
        train_scores.append(np.sqrt(mean_squared_error(y_tmp_train, model.predict(X_tmp_train))))
        valid_scores.append(np.sqrt(mean_squared_error(y_tmp_valid, model.predict(X_tmp_valid))))
    
    return train.scores.mean()

In [ ]:
optimizer = partial(objective, X=X_train, y=y_train)
study = optuna.create_study(direction='minimize')
study.optimize(optimizer, n_trial=100)

In [ ]:
study.best_param

In [ ]:
study.visualization.plot_optimization_history(study)

In [ ]:
study.visualization.plot_slice(study)